In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
IMAGE_SIZE = 256
NUM_CLASSES = 6

In [3]:
def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE], method='nearest')
        image = tf.cast(image, dtype=tf.float32)
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = image / 255
        image = tf.cast(image, dtype=tf.float32)
    return image

# GET TRAIN DATA

In [4]:
train_images_0 = []
for i in os.listdir('leftImg8bit\\train'):
    folder_path = 'leftImg8bit\\train\\' + i
    files = os.listdir(folder_path)
    for j in files:
        img_path = 'leftImg8bit\\train\\' + i + '\\' + j
        train_images_0.append(img_path)

train_images_0 = sorted(train_images_0)
print(len(train_images_0))

2975


In [5]:
train_masks_0 = []
for i in os.listdir('mask\\train'):
    train_masks_0.append('mask\\train\\' + i)

train_masks_0 = sorted(train_masks_0)
print(len(train_masks_0))

2975


# GET VALID DATA

In [6]:
val_images = []
for i in os.listdir('leftImg8bit\\val'):
    folder_path = 'leftImg8bit\\val\\' + i
    files = os.listdir(folder_path)
    for j in files:
        img_path = 'leftImg8bit\\val\\' + i + '\\' + j
        val_images.append(img_path)

val_images = sorted(val_images)
print(len(val_images))

500


In [7]:
val_masks = []
for i in os.listdir('mask\\val'):
    val_masks.append('mask\\val\\' + i)

val_masks = sorted(val_masks)
print(len(val_masks))

500


In [8]:
all_imgs = train_images_0 + val_images
all_masks = train_masks_0 + val_masks

# EVALUATE

In [9]:
model_DLv3_normal = keras.models.load_model('_Normal\\model_DLv3.h5')
model_DLv3_augment = keras.models.load_model('_Augmentation\\model_DLv3.h5')

In [10]:
model_Unet_normal = keras.models.load_model('_Normal\\model_Unet.h5')
model_Unet_augment = keras.models.load_model('_Augmentation\\model_Unet.h5')

In [11]:
from keras.metrics import MeanIoU

In [12]:
sum_IoU_0 = 0
sum_IoU_1 = 0
sum_IoU_2 = 0
sum_IoU_3 = 0

m = MeanIoU(num_classes=6)

for i in range(len(all_imgs)):
    print(i+1, end=' ') if ((i+1)%100 == 0) else print(end='')

    image_tensor = read_image(all_imgs[i])
    mask_tensor = read_image(all_masks[i], mask=True)
    mask_tensor = tf.squeeze(mask_tensor, axis=2)

    predictions_0 = model_DLv3_normal.predict(np.expand_dims((image_tensor), axis=0))  #(1,256,256,6)
    predictions_0 = tf.squeeze(predictions_0, axis=0) #(256,256,6)
    predictions_0 = tf.math.argmax(predictions_0, axis=2) 
    predictions_0 = tf.cast(predictions_0, dtype=tf.float32)
    m.update_state(mask_tensor, predictions_0)
    sum_IoU_0 += m.result().numpy()
    m.reset_state()

    predictions_1 = model_DLv3_augment.predict(np.expand_dims((image_tensor), axis=0))  #(1,256,256,6)
    predictions_1 = tf.squeeze(predictions_1, axis=0) #(256,256,6)
    predictions_1 = tf.math.argmax(predictions_1, axis=2) 
    predictions_1 = tf.cast(predictions_1, dtype=tf.float32)
    m.update_state(mask_tensor, predictions_1)
    sum_IoU_1 += m.result().numpy()
    m.reset_state()

    predictions_2 = model_Unet_normal.predict(np.expand_dims((image_tensor), axis=0))  #(1,256,256,6)
    predictions_2 = tf.squeeze(predictions_2, axis=0) #(256,256,6)
    predictions_2 = tf.math.argmax(predictions_2, axis=2) 
    predictions_2 = tf.cast(predictions_2, dtype=tf.float32)
    m.update_state(mask_tensor, predictions_2)
    sum_IoU_2 += m.result().numpy()
    m.reset_state()

    predictions_3 = model_Unet_augment.predict(np.expand_dims((image_tensor), axis=0))  #(1,256,256,6)
    predictions_3 = tf.squeeze(predictions_3, axis=0) #(256,256,6)
    predictions_3 = tf.math.argmax(predictions_3, axis=2) 
    predictions_3 = tf.cast(predictions_3, dtype=tf.float32)
    m.update_state(mask_tensor, predictions_3)
    sum_IoU_3 += m.result().numpy()
    m.reset_state()

100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 

In [14]:
print('# Average IoU score of DeepLab v3+ normal model: ', end='')
print(round(sum_IoU_0/len(all_imgs) * 100, 4), '%')
print('# Average IoU score of DeepLab v3+ augmentation model: ', end='')
print(round(sum_IoU_1/len(all_imgs) * 100, 4), '%')
print('\n# Average IoU score of UNET normal model: ', end='')
print(round(sum_IoU_2/len(all_imgs) * 100, 4), '%')
print('# Average IoU score of UNET augmentation model: ', end='')
print(round(sum_IoU_3/len(all_imgs) * 100, 4), '%')

# Average IoU score of DeepLab v3+ normal model: 67.4394 %
# Average IoU score of DeepLab v3+ augmentation model: 76.1968 %

# Average IoU score of UNET normal model: 68.0981 %
# Average IoU score of UNET augmentation model: 74.7571 %
